<a href="https://colab.research.google.com/github/HyeonhoonLee/TSA_pandas/blob/master/03_Time_Shifting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Shifting
때때로 시계열 데이터의 인덱스에 따라 데이터를 일정한 간격으로 이동시켜야 할 경우가 있습니다. 그렇게 time shifting 된 데이터를 lagged 데이터라고 하는데, 시계열 데이터 분석 알고리즘 대부분은 이러한 lagged 데이터를 필요로 합니다. pandas 에는 이를 지원하는 함수들이 있어 편리하게 사용할 수 있습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# csv 를 읽을 때부터 Date 칼럼을 인덱스로 지정
df = pd.read_csv('/content/drive/My Drive/Tacademy/TSA_pandas/data/apple_stock.csv', index_col='Date')

In [5]:
# 곧바로 index로 들어간 것을 알 수 있다.(단, 아직 object type이다.)
df.index

Index(['2009-12-31', '2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
       '2010-01-08', '2010-01-11', '2010-01-12', '2010-01-13', '2010-01-14',
       ...
       '2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20', '2018-12-21',
       '2018-12-24', '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-31'],
      dtype='object', name='Date', length=2265)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2265 entries, 2009-12-31 to 2018-12-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       2265 non-null   float64
 1   Low        2265 non-null   float64
 2   Open       2265 non-null   float64
 3   Close      2265 non-null   float64
 4   Volume     2265 non-null   float64
 5   Adj Close  2265 non-null   float64
dtypes: float64(6)
memory usage: 123.9+ KB


인덱스가 object 타입이므로 datetime 형으로 변환합니다.

In [7]:
# 01, 02 수업에서는 apply(pd.to_datetime)을 이용했는데, 이번에는 다른 방법으로 해본다.
## df['Date'] = df['Date'].apply(pd.to_datetime) ==> 이런식으로 했었음.
df.index = pd.to_datetime(df.index)

In [8]:
df.index

DatetimeIndex(['2009-12-31', '2010-01-04', '2010-01-05', '2010-01-06',
               '2010-01-07', '2010-01-08', '2010-01-11', '2010-01-12',
               '2010-01-13', '2010-01-14',
               ...
               '2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20',
               '2018-12-21', '2018-12-24', '2018-12-26', '2018-12-27',
               '2018-12-28', '2018-12-31'],
              dtype='datetime64[ns]', name='Date', length=2265, freq=None)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2265 entries, 2009-12-31 to 2018-12-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       2265 non-null   float64
 1   Low        2265 non-null   float64
 2   Open       2265 non-null   float64
 3   Close      2265 non-null   float64
 4   Volume     2265 non-null   float64
 5   Adj Close  2265 non-null   float64
dtypes: float64(6)
memory usage: 123.9 KB


In [10]:
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-12-31,30.478571,30.080000,30.447144,30.104286,88102700.0,20.159719
2010-01-04,30.642857,30.340000,30.490000,30.572857,123432400.0,20.473503
2010-01-05,30.798571,30.464285,30.657143,30.625713,150476200.0,20.508902
2010-01-06,30.747143,30.107143,30.625713,30.138571,138040000.0,20.182680
2010-01-07,30.285715,29.864286,30.250000,30.082857,119282800.0,20.145369


In [11]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-12-24,151.550003,146.589996,148.149994,146.830002,37169200.0,146.830002
2018-12-26,157.229996,146.720001,148.300003,157.169998,58582500.0,157.169998
2018-12-27,156.770004,150.070007,155.839996,156.149994,53117100.0,156.149994
2018-12-28,158.520004,154.550003,157.500000,156.229996,42291400.0,156.229996
2018-12-31,159.360001,156.479996,158.529999,157.740005,35003500.0,157.740005


## shift forward

In [12]:
# 만약 panda에 shift 기능이 없었다면, 아래의 3개 코드를 이용하여 shifting 했어야 한다.

In [16]:
import numpy as np
temp = np.asarray(df['Close'])

In [17]:
temp[:-1]

array([ 30.10428619,  30.5728569 ,  30.62571335, ..., 157.16999817,
       156.1499939 , 156.22999573])

In [18]:
temp[1:]
# 이제 값이 한칸 앞으로 간 것을 알 수 있다.

array([ 30.5728569 ,  30.62571335,  30.13857079, ..., 156.1499939 ,
       156.22999573, 157.74000549])

In [19]:
# 지금부터는 shift 함수를 이용해보자.
df.shift(1).head()
# shift하고나면 이전 데이터는 NaN으로 처리된다.

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,30.478571,30.080000,30.447144,30.104286,88102700.0,20.159719
2010-01-05,30.642857,30.340000,30.490000,30.572857,123432400.0,20.473503
2010-01-06,30.798571,30.464285,30.657143,30.625713,150476200.0,20.508902
2010-01-07,30.747143,30.107143,30.625713,30.138571,138040000.0,20.182680


In [20]:
# 인덱스 밖으로 밀려난 마지막 데이터를 잃게 됩니다.
df.shift(1).tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-12-24,158.160004,149.630005,156.860001,150.729996,95744600.0,150.729996
2018-12-26,151.550003,146.589996,148.149994,146.830002,37169200.0,146.830002
2018-12-27,157.229996,146.720001,148.300003,157.169998,58582500.0,157.169998
2018-12-28,156.770004,150.070007,155.839996,156.149994,53117100.0,156.149994
2018-12-31,158.520004,154.550003,157.500000,156.229996,42291400.0,156.229996


## shift backward

In [21]:
#거꾸로도 된다.
df.shift(-1).head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-12-31,30.642857,30.340000,30.490000,30.572857,123432400.0,20.473503
2010-01-04,30.798571,30.464285,30.657143,30.625713,150476200.0,20.508902
2010-01-05,30.747143,30.107143,30.625713,30.138571,138040000.0,20.182680
2010-01-06,30.285715,29.864286,30.250000,30.082857,119282800.0,20.145369
2010-01-07,30.285715,29.865715,30.042856,30.282858,111902700.0,20.279305


In [22]:
df.shift(-1).tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-12-24,157.229996,146.720001,148.300003,157.169998,58582500.0,157.169998
2018-12-26,156.770004,150.070007,155.839996,156.149994,53117100.0,156.149994
2018-12-27,158.520004,154.550003,157.500000,156.229996,42291400.0,156.229996
2018-12-28,159.360001,156.479996,158.529999,157.740005,35003500.0,157.740005
2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN


## shifting based on offset alias

In [25]:
# 한 달 앞으로 모든 데이터를 shift
## monthly 빈도(단위)로 6개월 앞으로 shift하고자 한다면,
df.tshift(freq='M', periods=6).head()
## 이런걸 lagging 한다고 한다.
## 이렇게하는 이유는 일부 결측치로 있는데 그냥 shift하게되면, index와 배열이 맞지 않기 때문.

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-06-30,30.478571,30.080000,30.447144,30.104286,88102700.0,20.159719
2010-06-30,30.642857,30.340000,30.490000,30.572857,123432400.0,20.473503
2010-06-30,30.798571,30.464285,30.657143,30.625713,150476200.0,20.508902
2010-06-30,30.747143,30.107143,30.625713,30.138571,138040000.0,20.182680
2010-06-30,30.285715,29.864286,30.250000,30.082857,119282800.0,20.145369


In [26]:
## panas 책에서는 periods 파라미터 없이 한꺼번에 할 수 있다고 한다.
df.tshift(freq='6M').head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-06-30,30.478571,30.080000,30.447144,30.104286,88102700.0,20.159719
2010-06-30,30.642857,30.340000,30.490000,30.572857,123432400.0,20.473503
2010-06-30,30.798571,30.464285,30.657143,30.625713,150476200.0,20.508902
2010-06-30,30.747143,30.107143,30.625713,30.138571,138040000.0,20.182680
2010-06-30,30.285715,29.864286,30.250000,30.082857,119282800.0,20.145369
